In [1]:
import mechanize
import http.cookiejar
from bs4 import BeautifulSoup
import re

import pandas as pd
import time

In [86]:
# Browser
br = mechanize.Browser()

# Cookie Jar
cj = http.cookiejar.LWPCookieJar()
br.set_cookiejar(cj)

# Browser options
br.set_handle_equiv(True)
br.set_handle_gzip(True)
br.set_handle_redirect(True)
br.set_handle_referer(True)
br.set_handle_robots(False)
br.set_handle_refresh(mechanize._http.HTTPRefreshProcessor(), max_time=1)

br.addheaders = [('User-agent', 'Chrome')]

In [95]:
# URL to load
url = 'https://www.cia.gov/the-world-factbook/references/guide-to-country-comparisons/'

# Loaded html
html = br.open(url).read()

# Read html with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

# Load country names
country_comparison_data = pd.read_excel('../Data/countries.xlsx', engine='openpyxl')

for link in soup.find_all(class_='link-button bold'):
    
    # update url
    url = 'https://www.cia.gov' + link.get('href')
    
    # Loaded html
    html = br.open(url).read()

    # Read html with Beautiful Soup
    soup = BeautifulSoup(html, 'html.parser')
    
    # Data title
    data_title = soup.find(class_='hero-title').text
    
    if data_title in country_comparison_data.columns:
        continue
    
    # List of countries
    countries = [value.text for value in soup.find_all(class_='text-button content-table-link')]

    # List of data
    elements = soup.find_all(class_='text-button content-table-link')
    data = [float(re.sub('[^\d\-.]', '', value.parent.next_sibling.text)) for value in elements]
    
    # Add data to countries dataframe
    df = pd.DataFrame({'country': countries, data_title: data})
    country_comparison_data = country_comparison_data.join(df.set_index('country'), on='Country')
    
    print('Loaded:', data_title)

Loaded: Area
Loaded: Population
Loaded: Median age
Loaded: Population growth rate
Loaded: Birth rate
Loaded: Death rate
Loaded: Net migration rate
Loaded: Maternal mortality ratio
Loaded: Infant mortality rate
Loaded: Life expectancy at birth
Loaded: Total fertility rate
Loaded: HIV/AIDS – adult prevalence rate
Loaded: Obesity – adult prevalence rate
Loaded: Alcohol consumption per capita
Loaded: Tobacco use
Loaded: Children under the age of 5 years underweight
Loaded: Education expenditures
Loaded: Youth unemployment rate (ages 15-24)
Loaded: Revenue from forest resources
Loaded: Revenue from coal
Loaded: Real GDP (purchasing power parity)
Loaded: Real GDP growth rate
Loaded: Real GDP per capita
Loaded: Inflation rate (consumer prices)
Loaded: Industrial production growth rate
Loaded: Labor force
Loaded: Unemployment rate
Loaded: Gini Index coefficient – distribution of family income
Loaded: Budget surplus (+) or deficit (-)
Loaded: Public debt
Loaded: Taxes and other revenues
Loaded:

In [105]:
pd.set_option('display.max_rows', 300)

In [115]:
country_comparison_data.to_excel('../Data/country_comparison_data.xlsx', index=False)